In [116]:
#gerekli kütüphaneler
import pandas as pd
import numpy as np
#veri okuma işlemi
df= pd.read_csv("datasets/nba.csv")
#gösterilecek max satır
pd.set_option('display.max_rows', 10)

In [117]:
#verilerin genel incelenmesi
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      457 non-null    object 
 1   Team      457 non-null    object 
 2   Number    457 non-null    float64
 3   Position  457 non-null    object 
 4   Age       457 non-null    float64
 5   Height    457 non-null    object 
 6   Weight    457 non-null    float64
 7   College   373 non-null    object 
 8   Salary    446 non-null    float64
dtypes: float64(4), object(5)
memory usage: 32.3+ KB


In [118]:
#sütun ve satır sayısının elde edilmesi
df.columns.size
df.index.size
#ilk 5 verinin alınması ".tail() son 5 veri"
df.head() 

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


In [119]:
# Takıma göre gruplayıp takımların oyuncularına ödediği ortalama, max, min ve kaç oyuncu olduğu bilgisine ulaşacağız
df_new=df.groupby("Team")["Salary"].agg([np.mean,np.max,np.min,np.sum])
#sayiya göre saydırıp oyuncu sayısını yeni bir sütun olarak ekliyoruz
df_new["PlayerNumber"]=df.groupby("Team")["Name"].count()

In [120]:
#Takımların belirli poziyonlardaki oyuncu sayısı
df.groupby(["Team","Position"])["Name"].count()
#Takıma göre pozisyonlardaki ortalama maaş,max ve min
df.groupby(["Team","Position"])["Salary"].agg([np.mean,np.max,np.min])

mean        amax       amin
Team               Position                                     
Atlanta Hawks      C         7.585417e+06  12000000.0  1000000.0
                   PF        5.988067e+06  18671659.0   947276.0
                   PG        4.881700e+06   8000000.0  1763400.0
                   SF        3.000000e+06   4000000.0  2000000.0
                   SG        2.607758e+06   5746479.0   525093.0
...                                   ...         ...        ...
Washington Wizards C         8.163476e+06  13000000.0   273038.0
                   PF        5.650000e+06   8000000.0  3300000.0
                   PG        9.011208e+06  15851950.0  2170465.0
                   SF        2.789700e+06   4662960.0   200600.0
                   SG        2.839248e+06   5694674.0   561716.0

[149 rows x 3 columns]

In [121]:
#Maaş verilerinde yer alan NaN değerleri ortalama değerler ile değiştirme ve Birden fazla değeri NaN olanları silme
df.dropna(subset=["Name"],how="any",inplace=True)
df_swNaN=df.dropna(subset=["Salary"],how="any",inplace=False)
#Nan değerlerini atmadan da aynı değeri vercektir sadece atma işlemine örnek olsun diye yapıldı
mean=df_swNaN["Salary"].mean()
df__a=df["Salary"].fillna(value=mean,inplace=True)


In [122]:
#filtre işlemleri:
# İsminde John ismi bulunan oyuncuları bulma

In [123]:
#Name string ifadesini ad-Soyad olarak ayırma işlemi 
df[["FirstName","LastName"]]=df["Name"].loc[df["Name"].str.split().str.len()==2].str.split(expand=True)
#Sütün sıralaması
df=df[["Name","FirstName","LastName","Team","Position","Age","Height","Weight","College","Salary"]]

#Name sütununa ihtiyacımız kalmadığından 
df.drop(axis=1,labels="Name",inplace=True)
#FirstName sütununa göre John işlemini arama
df.query("FirstName== 'John'")

,FirstName,LastName,Team,Position,Age,Height,Weight,College,Salary
2,John,Holland,Boston Celtics,SG,27.0,6-5,205.0,Boston University,4.842684e+06
126,John,Jenkins,Phoenix Suns,SG,25.0,6-4,215.0,Vanderbilt,9.813480e+05
216,John,Henson,Milwaukee Bucks,PF,25.0,6-11,229.0,North Carolina,2.943221e+06
382,John,Wall,Washington Wizards,PG,25.0,6-4,195.0,Kentucky,1.585195e+07


In [124]:
#25 yaşından büyüklerin yaşa göre kazanç ortalamaları
df[df["Age"]>=25].groupby("Age")["Salary"].mean()


Age
25.0    3.951130e+06
26.0    6.866566e+06
27.0    6.632008e+06
28.0    5.101559e+06
29.0    6.174838e+06
            ...     
36.0    2.238120e+06
37.0    1.079400e+07
38.0    1.840041e+06
39.0    2.517872e+06
40.0    4.666917e+06
Name: Salary, Length: 16, dtype: float64

In [125]:
#isminin içinde Je geçenler
#Kaç tane bozuk- kayıp veri olduğunun kontrolü
df["FirstName"].isna().sum()
df.dropna(subset=["FirstName"],how="all",inplace=True)
df[df["FirstName"].str.find("Je")!=-1]["FirstName"]

37     Jerian
50     Jerami
92       Jeff
95       Jeff
212    Jerryd
229    Jeremy
323      Jeff
333    Jeremy
335    Jeremy
456      Jeff
Name: FirstName, dtype: object

In [129]:
#işlenen verinin csv olarak kaydedilmesi
df.to_csv("Nba_yeni.csv")